<a href="https://colab.research.google.com/github/Kantdrav/tds/blob/main/GA4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q4

In [ ]:
import json                   # to convert API to json format

from urllib.parse import urlencode

import requests               # to get the webpage
from bs4 import BeautifulSoup # to parse the webpage

import pandas as pd
import re                     # regular expression operators

from datetime import datetime

We now GET the webpage of interest, from the server

In [ ]:
required_city = input("Enter the name of city: ")
location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
   'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
   's': required_city,
   'stack': 'aws',
   'locale': 'en',
   'filter': 'international',
   'place-types': 'settlement,airport,district',
   'order': 'importance',
   'a': 'true',
   'format': 'json'
})
location_url

Enter the name of city: Wellington


'https://locator-service.api.bbci.co.uk/locations?api_key=AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv&s=Wellington&stack=aws&locale=en&filter=international&place-types=settlement%2Cairport%2Cdistrict&order=importance&a=true&format=json'

In [ ]:
result = requests.get(location_url).json()
result

{'response': {'results': {'results': [{'id': '2179537',
     'name': 'Wellington',
     'container': 'New Zealand',
     'containerId': 2186224,
     'language': 'en',
     'timezone': 'Pacific/Auckland',
     'country': 'NZ',
     'latitude': -41.28664,
     'longitude': 174.77557,
     'placeType': 'settlement'},
    {'id': '2634573',
     'name': 'Wellington',
     'container': 'Telford and Wrekin',
     'containerId': 3333209,
     'language': 'en',
     'timezone': 'Europe/London',
     'country': 'GB',
     'latitude': 52.7,
     'longitude': -2.51667,
     'placeType': 'settlement',
     'topicId': 'cxrlk4yw0mjt'},
    {'id': '2634574',
     'name': 'Wellington',
     'container': 'Herefordshire',
     'containerId': 2647071,
     'language': 'en',
     'timezone': 'Europe/London',
     'country': 'GB',
     'latitude': 52.12753,
     'longitude': -2.74899,
     'placeType': 'settlement',
     'topicId': 'c90ek8r59xet'},
    {'id': '2634575',
     'name': 'Wellington',
     'con

In [ ]:
result['response']['results']['results'][0]['id']

'2179537'

In [ ]:
# url      = 'https://www.bbc.com/weather/1275339' # url to BBC weather, corresponding to a specific city (Mumbai, in this example)
url      = 'https://www.bbc.com/weather/'+result['response']['results']['results'][0]['id']
response = requests.get(url)

Next, we initiate an instance of  BeautifulSoup.

In [ ]:
soup = BeautifulSoup(response.content,'html.parser')

The information we want (daily high and low temp., and daily weather summary), are in specific blocks on the webpage.
We need to find the block type, type of identifier, and the identifier name (all these can be figured out by right clicking
on the webpage and selecting 'Inspect' on the Chrome browser; similar modus operandi for other browsers)

In [ ]:
daily_summary = soup.find('div', attrs={'class': 'wr-day-summary'})
daily_summary

<div class="wr-day-summary"><div class="gel-wrap"><span class="">Sunny and a moderate breeze</span><span class="wr-hide">Light rain showers and a fresh breeze</span><span class="wr-hide">Sunny and a moderate breeze</span><span class="wr-hide">Light cloud and a gentle breeze</span><span class="wr-hide">Sunny intervals and a moderate breeze</span><span class="wr-hide">Strong winds and rain</span><span class="wr-hide">Strong winds and light rain showers</span><span class="wr-hide">Light rain showers and a moderate breeze</span><span class="wr-hide">Drizzle and a gentle breeze</span><span class="wr-hide">Sunny intervals and a gentle breeze</span><span class="wr-hide">Light cloud and a moderate breeze</span><span class="wr-hide">Light cloud and light winds</span><span class="wr-hide">Sunny intervals and light winds</span><span class="wr-hide">Light rain and a gentle breeze</span></div></div>

In [ ]:
daily_summary.text

'Sunny and a moderate breezeLight rain showers and a fresh breezeSunny and a moderate breezeLight cloud and a gentle breezeSunny intervals and a moderate breezeStrong winds and rainStrong winds and light rain showersLight rain showers and a moderate breezeDrizzle and a gentle breezeSunny intervals and a gentle breezeLight cloud and a moderate breezeLight cloud and light windsSunny intervals and light windsLight rain and a gentle breeze'

General book keeping.

With the code snippet in the cell above, we get forecast data for 14 days, including today. We will now post process the data to first extract the required information/text and discard all the html wrapper code, then combine all variables into one common list, and finally convert it into a pandas data frame.

In [ ]:
daily_summary_list = re.findall('[a-zA-Z][^A-Z]*', daily_summary.text) #split the string on uppercase
daily_summary_list

['Sunny and a moderate breeze',
 'Light rain showers and a fresh breeze',
 'Sunny and a moderate breeze',
 'Light cloud and a gentle breeze',
 'Sunny intervals and a moderate breeze',
 'Strong winds and rain',
 'Strong winds and light rain showers',
 'Light rain showers and a moderate breeze',
 'Drizzle and a gentle breeze',
 'Sunny intervals and a gentle breeze',
 'Light cloud and a moderate breeze',
 'Light cloud and light winds',
 'Sunny intervals and light winds',
 'Light rain and a gentle breeze']

In [ ]:
datelist = pd.date_range(datetime.today(), periods=len(daily_summary_list)).tolist()
datelist

[Timestamp('2025-06-19 19:17:23.756823'),
 Timestamp('2025-06-20 19:17:23.756823'),
 Timestamp('2025-06-21 19:17:23.756823'),
 Timestamp('2025-06-22 19:17:23.756823'),
 Timestamp('2025-06-23 19:17:23.756823'),
 Timestamp('2025-06-24 19:17:23.756823'),
 Timestamp('2025-06-25 19:17:23.756823'),
 Timestamp('2025-06-26 19:17:23.756823'),
 Timestamp('2025-06-27 19:17:23.756823'),
 Timestamp('2025-06-28 19:17:23.756823'),
 Timestamp('2025-06-29 19:17:23.756823'),
 Timestamp('2025-06-30 19:17:23.756823'),
 Timestamp('2025-07-01 19:17:23.756823'),
 Timestamp('2025-07-02 19:17:23.756823')]

In [ ]:
datelist = [datelist[i].date().strftime('%Y-%m-%d') for i in range(len(datelist))]
datelist

['2025-06-19',
 '2025-06-20',
 '2025-06-21',
 '2025-06-22',
 '2025-06-23',
 '2025-06-24',
 '2025-06-25',
 '2025-06-26',
 '2025-06-27',
 '2025-06-28',
 '2025-06-29',
 '2025-06-30',
 '2025-07-01',
 '2025-07-02']

In [ ]:
d = dict()
for i in range(len(datelist)):
  d[datelist[i]] = daily_summary_list[i]
d

{'2025-06-19': 'Sunny and a moderate breeze',
 '2025-06-20': 'Light rain showers and a fresh breeze',
 '2025-06-21': 'Sunny and a moderate breeze',
 '2025-06-22': 'Light cloud and a gentle breeze',
 '2025-06-23': 'Sunny intervals and a moderate breeze',
 '2025-06-24': 'Strong winds and rain',
 '2025-06-25': 'Strong winds and light rain showers',
 '2025-06-26': 'Light rain showers and a moderate breeze',
 '2025-06-27': 'Drizzle and a gentle breeze',
 '2025-06-28': 'Sunny intervals and a gentle breeze',
 '2025-06-29': 'Light cloud and a moderate breeze',
 '2025-06-30': 'Light cloud and light winds',
 '2025-07-01': 'Sunny intervals and light winds',
 '2025-07-02': 'Light rain and a gentle breeze'}

In [ ]:
import json
json.dumps(d)

'{"2025-06-19": "Sunny and a moderate breeze", "2025-06-20": "Light rain showers and a fresh breeze", "2025-06-21": "Sunny and a moderate breeze", "2025-06-22": "Light cloud and a gentle breeze", "2025-06-23": "Sunny intervals and a moderate breeze", "2025-06-24": "Strong winds and rain", "2025-06-25": "Strong winds and light rain showers", "2025-06-26": "Light rain showers and a moderate breeze", "2025-06-27": "Drizzle and a gentle breeze", "2025-06-28": "Sunny intervals and a gentle breeze", "2025-06-29": "Light cloud and a moderate breeze", "2025-06-30": "Light cloud and light winds", "2025-07-01": "Sunny intervals and light winds", "2025-07-02": "Light rain and a gentle breeze"}'

---
# Q5
---


In [ ]:
from geopy.geocoders import Nominatim

locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode("Riyadh", exactly_one=False)
location
# x = min(location.raw['boundingbox'][0], location.raw['boundingbox'][1])
# round(float(x), 4)

[Location(Riyadh الرياض, Nouakchott نواكشوط, Riyadh, Nouakchott Sud نواكشوط الجنوبية, Mauritanie موريتانيا, (18.0107143, -15.955326, 0.0)),
 Location(منطقة الرياض, السعودية, (23.333333, 45.333333, 0.0)),
 Location(Riyadh, Nouakchott Sud نواكشوط الجنوبية, Mauritanie موريتانيا, (17.9909945, -15.9313582, 0.0))]

---
# Q6
---

In [ ]:
!pip install bs4

In [ ]:
import requests
from bs4 import BeautifulSoup as bs4
import re
points = "55" # @param {"type":"string","placeholder":"Enter points"}
query = "Rust" # @param {"type":"string","placeholder":"Enter query"}
resp = requests.get(f"https://hnrss.org/newest?q={query}&points={points}")
parsed_xml = bs4(resp.text, features="xml")
parsed_items = parsed_xml.find_all("item")
parsed_item = parsed_items[0]
description = parsed_item.find("description")
url_match = re.search(r'href="(.*?)"', str(description))
if url_match:
  first_url = url_match.group(1)
  print(first_url)
else:
  print("No URL found in the description.")


https://blog.vxn.dev/rou2exos-rusted-edition


---
# Q7
---

In [ ]:
import requests
from datetime import datetime

url = "https://api.github.com/search/users"

loc = "Singapore" # @param {"type":"string","placeholder":"Enter the location"}
followers = "190" # @param {"type":"string","placeholder":"Enter the number of followers"}

params = {
    "q": f"location:{loc} followers:>{followers}"
}

response = requests.get(url, params=params)
profiles = []
if response.status_code == 200:
    data = response.json()
    items = data.get("items", [])
    for item in items:
      profiles.append(item['url'])

datelist = []
cutoff = "2025-06-19T19:13:27Z"
c = datetime.strptime(cutoff, "%Y-%m-%dT%H:%M:%SZ")
for p in profiles:
  response = requests.get(p)
  data = response.json()
  date = datetime.strptime(data['created_at'], "%Y-%m-%dT%H:%M:%SZ")
  if date <= c:
    datelist.append(date.strftime("%Y-%m-%dT%H:%M:%SZ"))

max(datelist)

'2022-09-06T07:39:53Z'

---
# Q10
---

In [ ]:
!pip install -q pymupdf4llm
!apt-get -qq update && apt-get -qq install nodejs npm
!npm install -g prettier@3.4.2

import os
from google.colab import files
import pymupdf4llm
import subprocess
from IPython.display import Markdown, display

def run_command(command):
    """Helper function to run shell commands and print output/errors."""
    try:
        result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True, encoding='utf-8')
        # print("STDOUT:", result.stdout) # Optional: for debugging
        return result.stdout
    except subprocess.CalledProcessError as e:
        print(f"Error executing command: {command}")
        print("STDERR:", e.stderr)
        raise

# --- 1. Upload PDF ---
print("Please upload your PDF file (e.g., q-pdf-to-markdown.pdf)")
uploaded = files.upload()

if not uploaded:
    print("No file was uploaded. Exiting.")
else:
    pdf_filename = list(uploaded.keys())[0]
    pdf_filepath = os.path.join(os.getcwd(), pdf_filename)

    # Save the uploaded file to disk
    with open(pdf_filepath, 'wb') as f:
        f.write(uploaded[pdf_filename])
    print(f"\nUploaded '{pdf_filename}' successfully.")

    # --- 2. Convert PDF to Markdown using PyMuPDF4LLM ---
    raw_md_filename = "output_raw.md"
    raw_md_filepath = os.path.join(os.getcwd(), raw_md_filename)

    print(f"\nConverting '{pdf_filename}' to Markdown using PyMuPDF4LLM...")
    try:
        # Set PYTHONUTF8=1 environment variable for the conversion context
        # This is recommended by the documentation for pymupdf4llm
        env = os.environ.copy()
        env['PYTHONUTF8'] = '1'

        markdown_content = pymupdf4llm.to_markdown(pdf_filepath)

        with open(raw_md_filepath, "w", encoding="utf-8") as f:
            f.write(markdown_content)
        print(f"Raw Markdown saved to '{raw_md_filename}'")
    except Exception as e:
        print(f"Error during PDF to Markdown conversion: {e}")
        # If conversion failed, we stop here
        raise

    # --- 3. Format the Markdown using Prettier ---
    formatted_md_filename = "output_formatted.md"
    formatted_md_filepath = os.path.join(os.getcwd(), formatted_md_filename)

    print(f"\nFormatting '{raw_md_filename}' with Prettier v3.4.2...")
    try:
        # Prettier usually writes to the same file or stdout.
        # We'll read the raw, format it, and then get the content.
        # Option 1: Format in place and read
        # run_command(f"prettier --write {raw_md_filepath} --parser markdown")
        # with open(raw_md_filepath, "r", encoding="utf-8") as f:
        #     final_markdown_content = f.read()

        # Option 2: Pipe to prettier and capture output (cleaner for getting content directly)
        # Create the prettier command
        prettier_command = f"prettier --parser markdown" # Removed --write to get output to stdout

        # Run prettier, feeding the raw markdown content to its stdin
        process = subprocess.run(
            prettier_command,
            input=markdown_content,  # Pass the content directly
            capture_output=True,
            text=True,
            shell=True,
            check=True,
            encoding='utf-8'
        )
        final_markdown_content = process.stdout

        # Save the formatted content
        with open(formatted_md_filepath, "w", encoding="utf-8") as f:
            f.write(final_markdown_content)
        print(f"Formatted Markdown saved to '{formatted_md_filepath}'")

    except Exception as e:
        print(f"Error during Prettier formatting: {e}")
        print("Displaying raw Markdown instead (if available).")
        if os.path.exists(raw_md_filepath):
            with open(raw_md_filepath, "r", encoding="utf-8") as f:
                final_markdown_content = f.read()
        else:
            final_markdown_content = "Error: Could not produce Markdown."
            raise

    # --- 4. Submit (Display) the Formatted Markdown ---
    print("\n--- Final Formatted Markdown Output ---")
    # For Colab, using IPython.display.Markdown renders it nicely if it's valid Markdown.
    # Otherwise, just print the string.
    display(Markdown(final_markdown_content))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 15.9 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Extracting templates from packages: 100%
Selecting previously unselected package gyp.
(Reading database ... 126319 files and directories currently installed.)
Preparing to unpack .../000-gyp_0.1+20210831gitd6c5dd5-5_all.deb ...
Unpacking gyp (0.1+20210831gitd6c5dd5-5) ...
Selecting previously unselected package javascript-common.
Preparing to unpack .../001-javascript-common_11+nmu1_all.deb ...
Unpacking javascript-common (11+nmu1) ...
Selecting previously unselected package libjs-events.
Preparing to unpack .../002-libjs-events_3.3.0+~3.0.0-2_all.deb ...
Unpacking libjs-events (3.3.0+~3.0.0-2) ...
Selecting previously unselected package libjs-highlight.js.
Preparing to unpack .../003-libjs-highlight.js_9.18.5+dfsg1-1_all.

Saving q-pdf-to-markdown.pdf to q-pdf-to-markdown.pdf

Uploaded 'q-pdf-to-markdown.pdf' successfully.

Converting 'q-pdf-to-markdown.pdf' to Markdown using PyMuPDF4LLM...
Raw Markdown saved to 'output_raw.md'

Formatting 'output_raw.md' with Prettier v3.4.2...
Error during Prettier formatting: Command 'prettier --parser markdown' returned non-zero exit status 1.
Displaying raw Markdown instead (if available).

--- Final Formatted Markdown Output ---


**admitto** **pauper** **vinculum**

admoneo spero tot

aeger benevolentia cupiditas

alo tui adfectus

officiis super ait

benevolentia cuppedia stips

Conatus speciosus subseco alius aedificium doloremque cunabula terreo solutio. Audax arceo claustrum. Ambitus demens laboriosam ratione utpote conspergo stillic

cruentus bos

Solitudo tumultus tres
```
Vaco contabesco arx cattus coma.
Adnuo supellex admoveo sopor dolore audeo.
Solum thorax vulticulus thalassinus similique constans argumentum crustulum depraedor.
Stella comburo sui commodi.
```

  - demitto versus perspiciatis copia

  - suus veniam abeo tametsi

Conduco conitor acer vulgaris velociter.

  - summopere cariosus ubi explicabo

  - adficio sponte

  - vivo beatae

  - suffoco stultus crapula desparatus

  - correptius audentia recusandae

Aliqua ut amplexus deprimo .

  - civis depopulo tardus autem clam

  - caecus arbitro

  - apud avaritia atqui placeat

  - vomer audentia valde

  - vos vivo

  - correptius condico supra

  - doloribus supellex territo delectus

**pecco** **crur**

umquam comedo

despecto spoliatio

stabilis demo

thesauruscasso

color demoror
```
Impedit solio sol vox thorax harum pecus.
Adversus decens nemo cupiditas approbo adsum.
Catena canto agnosco tertius aeneus depraedor vomica aufero.
Damno versus pauper vae triumphus ventus sapiente crux modi vulnus.
Antea congregatio torrens corrupti aperte.
```

  - earum talus quis doloremque

  - dolores nesciunt tempus

  - umerus adflicto

  - repellendus voco neque statim

  - caritas solio soluta

  - ulciscor universe nisi

  - absque valde rem charisma

Comburo nobis libero. Tergo velociter animi suffoco. Cimentarius iusto utrum articulus solitudo autus calco apto cresco.

Acidus beatae depopulo fugit quo acies.
```
Cum bellum terminatio reiciendis vallum capio accommodo adopto testimonium ademptio.
Aeternus sperno concido alo colo.
```

Cras tenax calcar tabella crapula thesaurus cultura peccatus addo. Comprehendo sunt cornu tantum sublime verus in. Delego decimus cupressus ipsum villa tepesco c

**doloremque spernocariosustorrens suus**

torqueo vesco tamisium tenus aufero

aegre tot cernuus tamisium cariosus

  - admiratio capio doloremque dedecor

  - solvo teneo

  - voluptatibus anser cavus

  - ipsum unde similique

  - degenero abbas approbo apparatus adfectus

  - congregatio adfectus depraedor

  - iure alioqui

**tondeo credo** **vobis** **non** **colo**

sunt temporibus velit varietas callide

arceo complectus soluta creptio repudiandae

tremo cena tondeo conscendosursum

sum eaque templum adhuc cribro

summa vel infit illo chirographum

